In [2]:
from collections import Counter
import pandas as pd
import numpy as np
from fuzzywuzzy import process
import re

In [14]:
df_bank = pd.read_csv('bank_statement.csv')
df_ckt = pd.read_csv('checkout.csv')

In [15]:
common_bank = [word[0] for word in Counter(" ".join(df_bank["desc"]).split()).most_common(15)]
common_ckt = set([word[0] for word in Counter(" ".join(df_ckt["buyer_name"]).split()).most_common(300)])

common_bank = [word for word in common_bank if word != '.']

for i, word in enumerate(common_bank):
    if '.' == word:
        common_bank[i] = ''

    common_bank[i] = re.sub('\.', '\.', word)
    common_bank[i] = re.sub('\+', '\+', word)
    common_bank[i] = re.sub('\*', '\*', word)

In [16]:
df_bank['desc'] = df_bank['desc'].str.replace('\.', '').str.replace("|".join(common_bank), '', regex=True).str.strip().str.split().apply(lambda x: ' '.join(x))

In [17]:
df_bank['desc']

0                     BW CGNBWI YIWHI LEZC HX ZLLEZC HX
1                                    WOOG CRIZ NIWSI YO
2                             ZOMZW MBIX TM BUGW KZBUGW
3                     GO L/F HH PUYIM YEBF GN BYYEBF GN
4                     FBZUN WIQT JIW F TEGF PY TTEGF PY
                              ...                      
239995                          RV WC XUH QLIX GIXYI WX
239996           RO M/B EYEKI T GXZMIB YELB RX YYELB RX
239997               PUZ JTITI WJJIZJ TEDW HZ DTTEDW HZ
239998    WUVIJ XJVIS FISD SURWISZ POYER KSURWISZ POYER
239999             WEE BIJ SUI BCC QD LV XLDZ XBLV XLDZ
Name: desc, Length: 240000, dtype: object

In [18]:
max_statement = df_bank["stmt_amount"].max()
min_statement = df_bank["stmt_amount"].min()
group_bank = df_bank.sort_values(["stmt_amount"],ascending=True).groupby(["stmt_amount"])
group_ckt = df_ckt.sort_values(["ckt_amount"],ascending=True).groupby(["ckt_amount"])

In [31]:
def fuzzy_merge(df_1, df_2, key1, key2, thresh=40, limit=1):
    query = df_2[key2].tolist()
    index = df_2['ckt_id'].tolist()

    mapper = dict()

    for q, i in zip(query,index):
        mapper[q] = i
    
    match = df_1[key1].apply(lambda x: process.extract(x, query, limit=limit))
    df_1['matches'] = match
    
    filtered_match = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x]))
    ckt_id = filtered_match.apply(lambda x: mapper[x])

    df_1['matches'] = filtered_match
    df_1['ckt_id'] = ckt_id
    
    
    return df_1

In [41]:
from tqdm import tqdm

In [42]:
total = 0
final_df = pd.DataFrame()
for bank, ckt in zip( tqdm(group_bank) , group_ckt):
    # kamus name checkout, fuzzywuzzy ama bank
    res = fuzzy_merge(bank[1], ckt[1], 'desc', 'buyer_name')

    res.drop(['stmt_amount', 'desc', 'matches'], inplace=True, axis=1)
    final_df = pd.concat([final_df, res])

    # klo dpt match 1 ato 2 lgsg join aja

print(total)

100%|██████████| 18340/18340 [22:29<00:00, 13.59it/s] 0



In [44]:
final_df = final_df.sort_values(by=['stmt_id'])

In [45]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  

final_df.to_csv('out.zip', index=False,compression=compression_opts)